## load library

In [ ]:
from helper import *
import sys

## load config

In [ ]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)

# display(minio_config.__len__())
# display(minio_config)

## set variables

In [ ]:
## set api_config variables
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
topic = "service_opd_screening"
api_config["headers"] = headers
api_config["topic"] = topic

try:
  r = get_process(**api_config)
except Exception as ex:
  print(ex)
  sys.exit(0)

## set config to processing 
hospcode = r["hospcode"]
process_id = r["process_id"]
api_config["process_id"] = process_id
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/hisgw/{}/{}/{}".format(hospcode, topic,process_id)
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

# display(r.__len__())
# display(r)


## load stdcode hospital

In [ ]:
hospital = pd.read_csv(f"{std_code_path}/", engine="c",
                       dtype=pd.StringDtype(), storage_options=minio_config)


## load person

In [ ]:
person = read_csv(minio_config, path=source_path, filename="person.csv.gz")

# display(person.__len__())
# display(person)

## merge localcode to person

In [ ]:
person_merge_local = dict(
    title=["title_code", "code"],
    gender=["gender", "code"],
    rh_blood_group=["rh_blood_group", "code"],
    blood_group=["blood_group", "code"],
    nationality=["nationality_code", "code"],
    marital_status=["marital_status", "code"],
)
person = merge_codes(minio_config, person, local_code_path,
                     prefix_type="local", merge_config=person_merge_local)

# display(person.__len__())
# display(person)

## merge stdcode to person

In [ ]:
person_merge_std = dict(
    titles=["title_localcode", "localcode"],
    genders=["gender_localcode", "localcode"],
    rh_blood_groups=["rh_blood_group_localcode", "localcode"],
    blood_groups=["blood_group_localcode", "localcode"],
    nationalities=["nationality_localcode", "localcode"],
    marital_statuses=["marital_status_localcode", "localcode"],
)
person = merge_codes(minio_config, person, std_code_path,
                     prefix_type="std", merge_config=person_merge_std)
# person = person.drop_duplicates(subset=["hn"])

# display(person.__len__())
# display(person)

## rename columns person

In [ ]:
cols_rename = dict(
    group="groups",
    nationality="nationalities",
    status="statuses",
    gender="genders",
    title="titles"
)
person = rename_columns(person, cols_rename)

# display(person.__len__())
# display(person)

## load service_opd

In [ ]:
service_opd = read_csv(
    minio_config, path=source_path, filename="service_opd.csv.gz")

# try:
#     service_opd = read_csv(
#         minio_config, path=source_path, filename="service_opd.csv.gz")
# except Exception as ex:
#     print(ex)

# display(service_opd.__len__())
# display(service_opd)

## merge local service_opd

In [ ]:
service_opd_merge_local = dict(
    main_department=["main_department", "code"],
    sub_department=["sub_department", "code"],
)
service_opd = merge_codes(minio_config, service_opd, local_code_path,
                     prefix_type="local", merge_config=service_opd_merge_local)

# display(service_opd.__len__())
# display(service_opd)

## merge std service_opd

In [ ]:
service_opd_merge_std = dict(
    main_departments_opd=["main_department", "localcode"],
    # sub_department=["sub_department", "code"],
)
service_opd = merge_codes(minio_config, service_opd, std_code_path,
                          prefix_type="std", merge_config=service_opd_merge_std)

# display(service_opd.__len__())
# display(service_opd)

## rename service_opd columns

In [ ]:
cols_rename = dict(
    department = "departments_opd",
)
service_opd = rename_columns(service_opd, cols_rename)

# display(service_opd.__len__())
# display(service_opd)

## load service_opd_screening
- ไม่มี localcode && stdcode

In [ ]:
service_opd_screening = read_csv(minio_config, path=source_path, filename="service_opd_screening.csv.gz")

# display(service_opd_screening.__len__())
# display(service_opd_screening)

## merge localcode to service_opd_screening

In [ ]:
## service_opd_screening doesn,t have localcode

## merge stdcode to service_opd_screening

In [ ]:
# service_opd_screening doesn,t have stdcode

## merge service_opd_screeing to service_opd

In [ ]:
service_opd_screening = pd.merge(service_opd_screening, service_opd, how="left", left_on="vn", right_on="vn", suffixes=("", "_right"))
service_opd_screening = service_opd_screening[[a for a in service_opd_screening.columns.to_list() if a.endswith("_right")==False]]

# display(service_opd_screening.__len__())
# display(service_opd_screening)

## merge person to service_opd_screening

In [ ]:
service_opd_screening = pd.merge(
    service_opd_screening, person, how="left", left_on="hn", right_on="hn", suffixes=("", "_right"))
service_opd_screening = service_opd_screening[[
    a for a in service_opd_screening.columns.to_list() if a.endswith("_right") == False]]

# display(service_opd_screening.__len__())
display(service_opd_screening)

## set export status variable
- ใช้ตรวจสอบสถานะการ export package สำหรับส่งไปยัง minio

In [ ]:
is_success = dict(
  person_telephone = False,
  person = False,
  person_death = False,
  service_opd = False,
  service_opd_screening = False
)

## export to person_telephone

In [ ]:
# person_telephone = set_package(service_opd_screening, ["cid"], ["telephone", "type", "ext"])
person_telephone = dict()
person_telephone["cid"] = service_opd_screening["cid"]
person_telephone["telephone"] = ""
person_telephone["type"] = ""
person_telephone["ext"] = ""

person_telephone = pd.DataFrame(person_telephone)
is_success["person_telephone"] = upload_minio(
    minio_config, person_telephone, f"{destination_path}/person_telephone.csv.gz", **api_config)

# display(service_opd_screening.__len__())
# display(person_telephone)

In [ ]:
display(person)

## export to person

In [ ]:
# person = set_package(service_opd_screening, ["cid", "birhtdate","gender_code","gender_name","rh_blood_group_code","rh_blood_group_name","blood_group_code","blood_group_name","title_code","title_name","first_name","middle_name","last_name","nationality_code","nationality_name","marital_status_code","marital_status_name"], [])
# is_success["person_telephone"] = upload_minio(
#     minio_config, person_telephone, f"{destination_path}/person.csv.gz", **api_config)

person = dict()
person["cid"] = service_opd_screening["cid"]
person["birthdate"] = service_opd_screening["birthdate"]
person["gender_code"] = service_opd_screening["gender_stdcode"]
person["gender_name"] = service_opd_screening["gender_stdname_th"]
person["rh_blood_group_code"] = service_opd_screening["rh_blood_group_stdcode"]
person["rh_blood_group_name"] = service_opd_screening["rh_blood_group_stdname_th"]
person["blood_group_code"] = service_opd_screening["blood_group_stdcode"]
person["blood_group_name"] = service_opd_screening["blood_group_stdname_th"]
person["title_code"] = service_opd_screening["title_stdcode"]
person["title_name"] = service_opd_screening["title_stdname_th"]
person["first_name"] = service_opd_screening["first_name"]
person["middle_name"] = service_opd_screening["middle_name"]
person["last_name"] = service_opd_screening["last_name"]
person["nationality_code"] = service_opd_screening["nationality_stdcode"]
person["nationality_name"] = service_opd_screening["nationality_stdname_th"]
person["marital_status_code"] = service_opd_screening["marital_status_stdcode"]
person["marital_status_name"] = service_opd_screening["marital_status_stdname_th"]
person = pd.DataFrame(person)

is_success["person"] = upload_minio(
    minio_config, person, f"{destination_path}/person.csv.gz", **api_config)

# display(person.__len__())
# display(person)

## export to person_death 

In [ ]:
person_death = dict()
person_death["cid"] = service_opd_screening["cid"]
person_death["date_of_death"] = ""
person_death["cause_of_death"] = ""
person_death["place_of_death"] = ""
person_death = pd.DataFrame(person_death)

is_success["person_death"] = upload_minio(
    minio_config, person_death, f"{destination_path}/person_death.csv.gz", **api_config)

# display(person_death.__len__())
# display(person_death)

## export to service_opd

In [ ]:
service_opd = dict()
service_opd["hospcode"] = hospcode
service_opd["hospname"] = hospcode

## export to service_opd_screening

## export to service_opd_diagnosis


## export to service_opd_drugs


## export to service_appointment

## export to service_ipd

## service_ipd_diagnosis

## service_ipd_drugs

## summary success

In [ ]:
is_success = dict(
    person_telephone=False,
    person=False,
    person_death=False,
    service_opd=False,
    service_opd_screening=False
)

if list(is_success.values()).count(False)>0:
  print("ERROR")
else:
  print("SUCCESS")

# display(is_success.__len__())
display(is_success)
